In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# 상태 설정
# State는 TypedDict를 상속한 딕셔너리 타입입니다.
# messages 키는 list 타입이어야 하며, 추가로 add_messages 라는 메타데이터 또는 검증 규칙이 부여되어 있습니다.
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
# ToolNode로 도구 노드 구축
# TavilySearchResults는 LangChain 커뮤니티 툴 중 하나
# 웹 검색하는 역할임. max_results=2 검색 결과를 최대 2개까지 가져오도록 세팅.
# tools_condition은 보통 어떤 조건에서 도구를 사용할지 결정하는 규칙이나 함수
# ToolNode는 전달받은 tools를 사용해 특정 작업을 수행
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

### ToolNode?
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

In [5]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot) # llm 실행할 chatbot
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot") # tools , chatbot 사이에 엣지 그어줌. 시작점과 끝
graph_builder.add_conditional_edges("chatbot", tools_condition) # tool 이 필요한 경우인지 판단

graph_builder.set_entry_point("chatbot") # 사용자의 질문을 chatbot 이 받을 수 있는 포인트
graph = graph_builder.compile()

In [6]:
# llm 이 실시간 검색을 필요로 하는 질문
# additional_kwargs 로 tool_calls 로 tavily_search_results_json 가 활용됨
graph.invoke({"messages": {"role": "user", "content": "지금 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='지금 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='c4fb546d-7498-48be-b539-c595932eaf86'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9tXpEleoc9dGjU6e9XWirwzU', 'function': {'arguments': '{"query":"현재 대한민국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1672b9d4-0ea0-4aed-aaf5-49cf27478f6d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '현재 대한민국 대통령'}, 'id': 'call_9tXpEleoc9dGjU6e9XWirwzU', 'type': 'tool_call

In [7]:
# llm 이 바로 답할 수 있는 질문
# tool_calls 가 활용되지않음.
graph.invoke({"messages": {"role": "user", "content": "마이크로소프트가 어떤 회사야?"}})

{'messages': [HumanMessage(content='마이크로소프트가 어떤 회사야?', additional_kwargs={}, response_metadata={}, id='e7ad5a5a-e9ae-4d9b-a278-57f01454c66f'),
  AIMessage(content='마이크로소프트(Microsoft)는 미국의 다국적 기술 기업으로, 컴퓨터 소프트웨어, 전자 기기, 개인용 컴퓨터 및 관련 서비스 개발에 주력하고 있습니다. 1975년에 빌 게이츠(Bill Gates)와 폴 앨런(Paul Allen)에 의해 설립된 이 회사는 세계에서 가장 큰 소프트웨어 기업 중 하나로 알려져 있습니다.\n\n가장 유명한 제품은 운영 체제인 Windows와 사무용 소프트웨어인 Microsoft Office입니다. 또한, Azure라는 클라우드 컴퓨팅 서비스, Xbox 게임 콘솔, Surface 태블릿과 같은 하드웨어 제품도 제공합니다.\n\n마이크로소프트는 기업 및 개인 사용자를 위한 다양한 솔루션과 서비스를 제공하며, 기술 혁신과 지속적인 연구 개발에 중점을 둡니다. 전 세계적으로 많은 사람들이 마이크로소프트의 제품과 서비스를 사용하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 196, 'prompt_tokens': 89, 'total_tokens': 285, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerpr